In [1]:
import os,sys,re,h5py
from statsmodels import robust
import numpy as np
import csv
import pandas as pd

In [5]:
#fl = '/data/yuxin/m1Psi/IVT_test/IVT_U_test/single_fast5/0/00070ead-5dfa-4835-a011-62393e2b25e1.fast5'

#read_id = fl.split('/')[-1].replace('.fast5','')
#read_id


fl = '/data/jiayi/IVT_tombo/normalU_tombo/single_fast5_U/0/001f40b7-d3f7-4530-849e-bb4d409b8dff.fast5'

read_id = fl.split('/')[-1].replace('.fast5','')
read_id



fast5_data = h5py.File(fl,'r')

# raw data
raw_data = list(fast5_data['Raw/Reads/'].values())[0]
raw_data = raw_data['Signal'][()]
raw_data

# tombo resquiggled data
corr_data = fast5_data['/Analyses/RawGenomeCorrected_000/BaseCalled_template/Events']
corr_attrs = dict(list(corr_data.attrs.items()))
corr_data = corr_data[()]

# extract corrected signal features
event_starts = corr_data['start'] + corr_attrs['read_start_rel_to_raw']
event_lengths = corr_data['length']
# event_norm_means = corr_data['norm_mean']
# event_stdevs = corr_data['norm_stdev']
event_bases = corr_data['base']

In [7]:
# feature extraction by motifs
# raw signals
kmer_filter = "[ACTG][ACTG]T[ACTG][ACTG]"
def extract_kmer_signals(idx):
    base0,base1,base2,base3,base4 = [event_bases[idx+x].decode() for x in [-2,-1,0,1,2]]
    kmer = "%s%s%s%s%s"%(base0,base1,base2,base3,base4)
    
    # kmer matching
    mismatches = [x.start() for x in re.finditer(kmer_filter, kmer)]
    if len(mismatches)==0:
        kmer_signals = np.array([])
        for x in [-2,-1,0,1,2]:
            kmer_signals = np.append(kmer_signals,[raw_data[event_starts[idx+x]:event_starts[idx+x]+event_lengths[idx+x]]])
        return(kmer_signals)

dataset = [extract_kmer_signals(idx) for idx in range(2,len(event_lengths)-3)]
print(dataset)
# dataset = np.concatenate((dataset, [kmer_signals]), axis=0)

[array([661., 687., 636., 616., 624., 620., 685., 697., 684., 688., 664.,
       707., 690., 663., 691., 685., 687., 698., 660., 684., 668., 678.,
       671., 636., 695., 688., 672., 680., 669., 667., 675., 713., 773.,
       745., 731., 759., 781., 668., 706., 690., 705., 685., 693., 696.,
       682., 714., 694., 691., 707., 677., 731., 693., 698., 714., 708.]), array([697., 684., 688., 664., 707., 690., 663., 691., 685., 687., 698.,
       660., 684., 668., 678., 671., 636., 695., 688., 672., 680., 669.,
       667., 675., 713., 773., 745., 731., 759., 781., 668., 706., 690.,
       705., 685., 693., 696., 682., 714., 694., 691., 707., 677., 731.,
       693., 698., 714., 708., 729., 685., 715., 718., 720., 652., 675.,
       704., 713., 673., 648., 684.]), array([678., 671., 636., 695., 688., 672., 680., 669., 667., 675., 713.,
       773., 745., 731., 759., 781., 668., 706., 690., 705., 685., 693.,
       696., 682., 714., 694., 691., 707., 677., 731., 693., 698., 714.,
       70

In [8]:

output = open('/data/jiayi/IVT_tombo/normalU_tombo/tombo_U_features.csv','w',encoding='utf-8')

kmer_filter = "[ACTG][ACTG]T[ACTG][ACTG]"
uniq_arr = np.unique(raw_data)
raw_data = (raw_data - np.median(uniq_arr)) / np.float(robust.mad(uniq_arr))
raw_data = raw_data.tolist()

for idx in range(2,len(event_lengths)-3):
    base0,base1,base2,base3,base4 = [event_bases[idx+x].decode() for x in [-2,-1,0,1,2]]
    kmer = "%s%s%s%s%s"%(base0,base1,base2,base3,base4)
    
    mismatches = [x.start() for x in re.finditer(kmer_filter, kmer)]
    if len(mismatches) == 0:
        continue
    kmer_signal = [raw_data[event_starts[idx+x]:event_starts[idx+x]+event_lengths[idx+x]]
                  for x in [-2,-1,0,1,2]]
    mean = [np.mean(kmer_signal)]
    std = [np.std(kmer_signal)]
    md = [np.median(kmer_signal)]
    length = [event_lengths[idx+x] for x in [-2,-1,0,1,2]]
    feature = mean+std+md+length 
    output.writelines(' '.join([str(x) for x in feature]))
    output.writelines('\n')
    
output.close()

/tmp/ipykernel_688422/172609180.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  raw_data = (raw_data - np.median(uniq_arr)) / np.float(robust.mad(uniq_arr))
/home/jiayi/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:164: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = asanyarray(a)
/home/jiayi/miniconda3/lib/python3.9/site-packages/numpy/core/_methods.py:198: VisibleDeprecationWarning: Creating an ndarray from ragged nest

TypeError: unsupported operand type(s) for /: 'list' and 'int'

In [9]:
dir = '/data/jiayi/IVT_tombo/normalU_tombo/fast5_fl_U.txt'

output_csv = open('/data/jiayi/IVT_tombo/normalU_tombo/tombo_U_features.csv','w',encoding='utf-8')
csv_writer = csv.writer(output_csv)


file_list = []
for i in open(dir,'r'):
    file_list.append(i.rstrip())

file_dir = file_list[0]
fast5_data = h5py.File(file_dir,'r')
    
raw_data = list(fast5_data['Raw/Reads/'].values())[0]
raw_data = raw_data['Signal'][()]
    
uniq_arr = np.unique(raw_data)
raw_data = (raw_data - np.median(uniq_arr)) / np.float(robust.mad(uniq_arr))
raw_data = raw_data.tolist()

corr_data = fast5_data['/Analyses/RawGenomeCorrected_000/BaseCalled_template/Events']
corr_attrs = dict(list(corr_data.attrs.items()))
coor_data = corr_data[()]
    
event_starts = corr_data['start'] + corr_attrs['read_start_rel_to_raw']
event_lengths = corr_data['length']
event_bases = corr_data['base']
    
seq_length = len(event_bases)
kmers_signal = []
for idx in range(2,seq_length-3):
    base0,base1,base2,base3,base4 = [event_bases[idx+x].decode() for x in [-2,-1,0,1,2]]
    kmer = "%s%s%s%s%s"%(base0,base1,base2,base3,base4)
    mismatches = [x.start() for x in re.finditer(kmer_filter,kmer)]
    if len(mismatches) == 0:
        continue
        
    kmer_signal = [raw_data[event_starts[idx+x]:event_starts[idx+x]+event_lengths[idx+x]] for x in [-2,-1,0,1,2]]
    mean = [np.mean(x) for x in kmer_signal]
    std = [np.std(x) for x in kmer_signal]
    md = [np.median(x) for x in kmer_signal]
    length = [event_lengths[idx+x] for x in [-2,-1,0,1,2]]
    print([idx] + [kmer] + mean + std + md + length)
    feature = pd.DataFrame([idx] + [kmer] + mean + std + md + length)

csv_writer.writerows(kmers_signal)

FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'single_fast5_U/33/8d41a41f-f94a-4e2b-8b44-bc9c85b7648e.fast5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [10]:
help(pd)

/home/jiayi/miniconda3/lib/python3.9/inspect.py:351: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  value = getattr(object, key)
/home/jiayi/miniconda3/lib/python3.9/inspect.py:351: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  value = getattr(object, key)
/home/jiayi/miniconda3/lib/python3.9/inspect.py:351: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  value = getattr(object, key)
IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
